In [2]:
import sys
!{sys.executable} -m pip install -qU langchain-huggingface



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:

pip install -U langchain langchain-core

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install sentence-transformers langchain_ollama owlready2

In [4]:

from owlready2 import get_ontology, Thing, Restriction, And, Or, Not
from langchain_core.documents import Document
from owlready2 import FunctionalProperty, InverseFunctionalProperty, TransitiveProperty, SymmetricProperty, AsymmetricProperty, ReflexiveProperty, IrreflexiveProperty

def ontology_to_vector_store(onto, llm, vector_store):

    #convert restriction into text
    def restriction_to_text(restriction):

        #get name of property the restriction applies to
        #if property is None fallback to string, this shouldnt happen, but just incase 
        prop_name = restriction.property.name if restriction.property else str(restriction.property)

        # Check for "exists" attribute
        if hasattr(restriction, "some_values_from") and restriction.some_values_from: #check attribute and not empty
            #get name from target class or fallback to object itself
            target = getattr(restriction.some_values_from, "name", restriction.some_values_from)
            return f"must have some {prop_name} from {target}"
        
        # Check for "all" attribute
        elif hasattr(restriction, "all_values_from") and restriction.all_values_from:
            target = getattr(restriction.all_values_from, "name", restriction.all_values_from)
            return f"can only have {prop_name} from {target}"
        
        # Check for "≥" attribute
        elif hasattr(restriction, "min_cardinality") and restriction.min_cardinality is not None: #add is not None, because "0" would be handled as False
            return f"{prop_name} ≥ {restriction.min_cardinality}"
        
        # Check for "≤" attribute
        elif hasattr(restriction, "max_cardinality") and restriction.max_cardinality is not None: #add is not None, because "0" would be handled as False
            return f"{prop_name} ≤ {restriction.max_cardinality}"
        
        #Fallback for other types of restrictions
        else:
            return f"{prop_name} with unknown restriction {restriction}"

    #recursive function that turns expression to text
    def expression_to_text(expr):

        #Ontology class
        if hasattr(expr, "name"):
            return expr.name
        
        #constraint in ontology
        elif isinstance(expr, Restriction):
            return restriction_to_text(expr)
        
        #And expression
        elif isinstance(expr, And):
            return " and ".join(expression_to_text(c) for c in expr.Classes)
        
        #Or expression
        elif isinstance(expr, Or):
            return " or ".join(expression_to_text(c) for c in expr.Classes)
        
        #Not expr
        elif isinstance(expr, Not):
            return f"not {expression_to_text(expr.Class)}"

        #Fallback, return string
        else:
            return str(expr)

    #turns ontology class to text
    def class_to_text(cls):
        lines = [f"The class {cls.name} is defined as follows:"]
        
        # Superclasses
        for parent in cls.is_a:
            lines.append(f"- Subclass of {expression_to_text(parent)}")
        
        # Equivalent classes
        for eq in cls.equivalent_to:
            lines.append(f"- Equivalent to {expression_to_text(eq)}")
        
        return "\n".join(lines)

    # turn object or data property to text
    def property_to_text(prop):
        #starting text
        lines = [f"The property {prop.name} is defined as follows:"]
        
        #check for "domain" attribute and not empty
        if hasattr(prop, "domain") and prop.domain:
            #convert each class in domain to text
            domain_text = ", ".join(expression_to_text(d) for d in prop.domain)
            lines.append(f"- Domain: {domain_text}")
        
        #check for "range" attribute and not empty
        if hasattr(prop, "range") and prop.range:
            #convert each class in range to text
            range_text = ", ".join(expression_to_text(r) for r in prop.range)
            lines.append(f"- Range: {range_text}")
        
        # Only include characteristics if they exist
        if issubclass(prop, FunctionalProperty):
            lines.append("- This property is functional (at most one value).")
        if issubclass(prop, InverseFunctionalProperty):
            lines.append("- This property is inverse functional (each value has at most one subject).")
        if issubclass(prop, TransitiveProperty):
            lines.append("- This property is transitive.")
        if issubclass(prop, SymmetricProperty):
            lines.append("- This property is symmetric.")
        if issubclass(prop, AsymmetricProperty):
            lines.append("- This property is asymmetric.")
        if issubclass(prop, ReflexiveProperty):
            lines.append("- This property is reflexive (every individual is related to itself).")
        if issubclass(prop, IrreflexiveProperty):
            lines.append("- This property is irreflexive (no individual is related to itself).")
        
        return "\n".join(lines)

    # turn instances to text
    def instance_to_text(instance):
        lines = [f"The instance {instance.name} is defined as follows:"]

        # Classes this instance belongs to
        types = [cls.name for cls in instance.is_a if hasattr(cls, "name")]
        if types:
            lines.append(f"- Belongs to class(es): {', '.join(types)}")

        # All properties (object and data) that this individual actually has
        for prop in instance.get_properties():
            try:
                values = getattr(instance, prop.python_name, [])
            except Exception:
                continue
            if values:
                # make iterable if not
                if not isinstance(values, (list, tuple, set)):
                    values = [values]
                # check if each value has a name (object property) or is literal (data property)
                values_text = ", ".join(getattr(v, "name", str(v)) for v in values)
                lines.append(f"- {prop.name}: {values_text}")

        return "\n".join(lines)

    # Collect all text for LLM
    ontology_texts = []

    # Add text for Classes
    for cls in onto.classes():
        #print(cls)
        ontology_texts.append(class_to_text(cls))

    # Add text for Object and datatype properties
    for prop in onto.object_properties():
        ontology_texts.append(property_to_text(prop))
    for prop in onto.data_properties():
        ontology_texts.append(property_to_text(prop))

    # Add text for instances
    for instance in onto.individuals():
        ontology_texts.append(instance_to_text(instance)) 

    # Turn into LLM documents
    docs = []
    for text in ontology_texts:
        prompt = (
            "Convert the following ontology snippet into clear, human-readable English.\n"
            f"Snippet: {text}\n\n"
            "Do not add external information or commentary. Only describe what is present. Do not include any \"Here is a snippet\" to the output"
        )
        human_text = llm.invoke(prompt)
        print(human_text.content)
        docs.append(Document(page_content=human_text.content))

    # Add to your vector store
    vector_store.add_documents(docs)
    return vector_store


In [5]:
def scenario_splitter_llm(story, llm):
    story_to_scenarios_prompt = f"""
    You are an assistant that extracts logical or factual scenarios from a given story.

    A scenario is a small, self-contained statement or short passage that expresses one or more closely related facts, events, or claims that can later be checked for correctness or consistency using an ontology.
    ---

    ### Guidelines:
    - Each scenario should capture a complete idea, including all details that are logically connected (for example, cause–effect, contrast, or relationship).
    - If two or more statements are relevant to each other (e.g., one qualifies, contradicts, or explains the other), combine them into one scenario.
    - Preserve contextual and relational details - who, what, where, when, and why.
    - Avoid redundancy - do not restate the same information.
    - Keep each scenario brief but complete (usually one to three sentences).
    - Include implicit facts when they are important (e.g., “John’s wife Amira” implies John is married to Amira).
    - Ensure that every piece of relevant information from the story appears in at least one scenario.

    ---

    Example

    Story:
    “John is 15 years old and is on vacation with his wife Amira in Italy. Their daughter Anna can’t wait to visit the Eiffel Tower.”

    Extracted Scenarios:
    1. John is 15 years old and is married to Amira.
    2. John and Amira are on vacation in Italy.
    3. John and Amira have a daughter named Anna.
    4. Anna wants to visit the Eiffel Tower.

    ---

    Now extract the scenarios from the following story and number them clearly:

    {story}
    """

    # Query the LLM
    response = llm.invoke(story_to_scenarios_prompt)
    if hasattr(response, "content"):  # LangChain AIMessage
        output_text = response.content
    elif isinstance(response, dict) and "content" in response:
        output_text = response["content"]
    elif isinstance(response, str):
        output_text = response
    else:
        raise TypeError(f"Unexpected LLM response type: {type(response)}")

    # Extract list items using regex
    import re
    scenarios = re.findall(r'(?:\d+\.\s*)(.+)', output_text)
    scenarios = [s.strip() for s in scenarios if s.strip()]

    # If the LLM doesnt number them, fallback to line splitting
    if not scenarios:
        scenarios = [line.strip("-• \t") for line in output_text.splitlines() if line.strip()]

    return scenarios


In [34]:
from typing import List, Dict, Any
from langchain_core.prompts import PromptTemplate

#fixes scenarios incrementally using ontology to reason. Ensures each fix remains consistent with previous scenario updates
def fix_scenarios_incrementally(scenarios: List[str], vector_store, llm, prompt_template: PromptTemplate, k):

    updated_scenarios = []
    accumulated_story = "" #Used as reasoning context for the next step
    results = []

    for i, scenario in enumerate(scenarios, start=1):
        #Retrieve ontology documents relevant to current scenario (RAG)
        retrieved_docs = vector_store.similarity_search(scenario, k=k)
        ontology_text = "\n\n".join(doc.page_content for doc in retrieved_docs)

        #Make context with ontology + story so far
        full_context = (
            f"Ontology context:\n{ontology_text}\n\n"
            f"Story so far:\n{accumulated_story.strip() or '(none so far)'}\n"
        )

        #Make the prompt using the template
        rendered_prompt = prompt_template.format(
            question=scenario.strip(),
            context=full_context.strip(),
        )

        #Invoke the model
        llm_response = llm.invoke(rendered_prompt)
        response_text = llm_response.content.strip()

        #Parse model output
        updated_scenario = scenario
        consistency = "Unknown"
        explanation_lines = []

        in_explanation = False

        for line in response_text.splitlines():
            lowered_line = line.lower().strip()
            if lowered_line.startswith("consistency:"):
                consistency = line.split(":", 1)[1].strip()#Split at first ":" only and take everything after it. Example: consistency: consistent, we only want 2nd part
                in_explanation = False
            elif lowered_line.startswith("updated scenario:"):
                updated_scenario = line.split(":", 1)[1].strip()#Split at first ":" only and take everything after it.
                in_explanation = False
            elif lowered_line.startswith("explanation:"):
                #begin collecting explanation text
                after = line.split(":", 1)[1].strip()#Split at first ":" only and take everything after it.
                if after:
                    explanation_lines.append(after)
                in_explanation = True
            #if just a line and last line was explanation. (This is next sentence of explanation)
            else:
                if in_explanation:
                    explanation_lines.append(line) #Add next sentence of explanation

        #Convert explanation into single string
        explanation = "\n".join(explanation_lines).strip()

        #If consistent, use unchanged scenario
        if(consistency.lower().strip() == "consistent"):
            #Update accumulated story so future reasoning uses this to fix scenarios
            accumulated_story = (accumulated_story + "\n" + scenario).strip()
            #Update scenario
            updated_scenarios.append(scenario)

        #If not consistent, use llm updated scenario
        else:
            #Update accumulated story so future reasoning uses this to fix scenarios
            accumulated_story = (accumulated_story + "\n" + updated_scenario).strip()
            #Update scenario
            updated_scenarios.append(updated_scenario)

        #Store result for logging or later evaluation
        results.append({
            "step": i,
            "original": scenario,
            "updated": updated_scenario,
            "consistency": consistency,
            "ontology_used": ontology_text.strip(),
            "llm_raw_output": response_text.strip(),
            "explanation": explanation,
            "story_so_far": accumulated_story.strip()
        })

    return updated_scenarios, results


In [35]:
from langchain_core.prompts import PromptTemplate

scenario_fixing_prompt = PromptTemplate.from_template("""
You are a reasoning assistant that checks whether each part of a story (called a 'scenario')
is logically consistent with (a) the ontology information provided and (b) the previously verified parts of the story.

You are given:
1. Ontology context - short human-readable snippets extracted from an ontology (classes, restrictions, domains/ranges, instances).
2. Story so far - previously verified or corrected parts of the story.
3. A new scenario - the next statement to evaluate.

Your tasks (use only the provided ontology context and story so far; do NOT use external world knowledge):
- Determine whether the new scenario is consistent with BOTH the ontology context and the story so far.
- If inconsistent, rewrite the scenario minimally to make it consistent.
- Explain, concisely and step-by-step, which ontology facts (from the provided context) you used and why the original scenario was inconsistent or consistent.
- If you changed the scenario, explain which minimal change you made and why that resolves the inconsistency.
- Prefer small, natural edits that preserve meaning and story flow.

**STRICT OUTPUT FORMAT (must follow exactly)**

Consistency: [Consistent / Inconsistent]

Updated scenario: [Corrected or unchanged scenario; if unchanged, copy the original. DO NOT OUTPUT "No change" or "unchanged"]

Explanation:
[A short, numbered or bulleted explanation (1-4 sentences) describing:
 - which ontology statements you used (quote or paraphrase them),
 - why the scenario is or is not consistent,
 - and, if inconsistent, why the updated scenario fixes the problem.]

Example:
Ontology context:
- Adult ⊆ Person; Person hasAge ≥ 0 and ≤ 110.
- Adult equivalent: Person and hasAge ≥ 18.
- Property isMarriedTo domain=Adult, range=Adult.

Story so far:
John is 15 years old.

Next scenario:
John is married to Amira.

Expected output:                                                   
Let's think step-by-step.
Explanation:
1. Ontology: "isMarriedTo domain=Adult" and "Adult ... hasAge ≥ 18". John is 15, so being married violates the domain constraint.
2. I changed John's age to 23 so he satisfies Adult and therefore the marriage relation is allowed. This is a minimal, natural fix preserving intended meaning.                                                    
Consistency: Inconsistent
Updated scenario: John is 23 years old and is married to Amira.

Now analyze using the provided context below.
{context}

Next scenario:
{question}
                                                      
Let's think step-by-step
""")


In [8]:
from langchain_core.prompts import PromptTemplate

fix_story_prompt = PromptTemplate.from_template("""
You are a reasoning assistant tasked with fixing inconsistencies in a story.

You are given:

The original story:
{original_story}
                                           
The original inconsistent scenarios/facts taken from the original story
{scenarios}

A list of updated, consistent scenarios/facts:
{updated_scenarios}

Your task is to rewrite the new updated scenarios into the original story. Make minimal changes to the original story; preserve the writing style, narrative flow, and character details as much as possible. 

Return only the fixed story.
""")

In [9]:
def fix_story(llm, story, scenarios, updated_scenarios):
    fixed_story = llm.invoke(fix_story_prompt.format(
    original_story=story,
    scenarios=scenarios,
    updated_scenarios="\n".join(updated_scenarios)
))
    return fixed_story.content

In [79]:
from typing import List, TypedDict, Optional, Dict, Any
from langgraph.graph import StateGraph, START, END
from owlready2 import get_ontology
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama

class StoryState(TypedDict, total=False):
    story: str
    ontology: Any
    scenarios: List[str]
    updated_scenarios: List[str]
    results: Any
    fixed_story: str
    llm: Any
    vector_store: Any
    loop_count: int #how many times check consistency node has been done

#Turns ontology into natural language docs for RAG
def vector_store_node(state: StoryState) -> Any:
    onto = state.get("ontology")
    vector_store = state.get("vector_store")
    vector_store = ontology_to_vector_store(onto, llm, vector_store)
    return {"vector_store": vector_store}

#Create scenarions based on story
def split_scenarios_node(state: StoryState) -> Dict[str, Any]:
    story = state.get("story", "")
    scenarios = scenario_splitter_llm(story, llm)
    return {"scenarios": scenarios}

#Fixes the scenarios incrementally
def check_consistency_node(state: StoryState) -> Dict[str, Any]:
    scenarios = state.get("updated_scenarios") or state.get("scenarios")
    vector_store = state.get("vector_store")
    llm = state.get("llm",)
    loop_count = state.get("loop_count", 0)
    updated_scenarios, results = fix_scenarios_incrementally(scenarios, vector_store, llm, scenario_fixing_prompt, k=5)
    return {"updated_scenarios" : updated_scenarios, "results": results, "loop_count": loop_count + 1}

#Fixes the story
def fix_story_node(state: StoryState) -> Dict[str, Any]:
    story = state.get("story", "")
    scenarios = state.get("scenarios")
    updated_scenarios = state.get("updated_scenarios")
    fixed_story = fix_story(llm, story, scenarios, updated_scenarios)

    return { "fixed_story": fixed_story}

#condition for looping consistency check
def should_repeat_check_consistency(state: StoryState):
    #Limit to N loops
    if state["loop_count"] < 2:#Do total of 2 times
        return "repeat"
    return "done"

builder = StateGraph(StoryState)
builder.add_node("vector_store", vector_store_node)
builder.add_node("split_scenarios", split_scenarios_node)
builder.add_node("check_consistency", check_consistency_node)
builder.add_node("fix_story", fix_story_node)

builder.add_edge(START, "split_scenarios")
builder.add_edge(START, "vector_store")
builder.add_edge("vector_store", "check_consistency")
builder.add_edge("split_scenarios", "check_consistency")
builder.add_conditional_edges(
    "check_consistency",
    should_repeat_check_consistency,
    {
        "repeat": "check_consistency", #loop
        "done": "fix_story",           #continue
    },
)
#builder.add_edge("check_consistency", "fix_story")
builder.add_edge("fix_story", END)

graph = builder.compile()

onto = get_ontology("./Ontology_Assignment.rdf").load()

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)


#-------------------------------------------------
#   Stories. 
#   Comment out all other stories you dont use.
#-------------------------------------------------

#Story 1
#story = """John is 15 years old and is on vacation with his wife Amira in Italy. Their daughter Anna can’t wait to visit the famous landmark of Florance, the Eiffel Tower. After that they will go out to eat. John suggests they eat pizzas since Italy is famous for them, they will eat at restaurant Riccolo located in Florance.
#Since Anna has a vitamin C deficiency she will order a pizza that contains tomatoes for vitamin C, so she will get the pizza bianca. John will get the classic Margherita pizza and Amira orders a pepperoni pizza.
#They sit by the window of the small restaurant the air filled with the smell of garlic and baking dough. Anna swings her legs impatiently under the table, still talking about the Eiffel Tower, while Amira flips through a guidebook about Florance.
#15 minutes later the farmer named Leo from the restaurant brings their pizzas and they eat, and Anna says to John how cool it is that the waiter is from France."""

#Story 2
story = "Jack and his family are celebrating his birthday, Jack just turned 235 years old. Jack loves talking about himself and everything he has done in his life, thats why many people would consider Jack a reserved person. This year, Jack’s family decided to visit the beautiful city of Quito, famous for its landmark La Virgen del Panecillo, which overlooks the city from a tall hill. Since the city doesn’t have mountainous terrain and is easy to walk through, Jack enjoys strolling through the streets without any trouble. It’s quite convenient, because Jack can’t walk very far due to his asthma. Luckily, Jack brought his medication for his asthma, so whenever he’s out of breath, he can simply take his antihistamine pills."

#Story 3
#story = """Emily and her son Michael decided to adopt a cat from the local shelter in Denver, a walkable city surrounded by tall mountains. The city’s most famous landmark, the Red Rocks Amphitheatre, could be seen from their apartment window. Michael was thrilled to finally have a pet to take on walks through the flat city streets. The shelter worker handed them a small brown dog named Whiskers, explaining that he meows loudly when he wants food.
#Later that afternoon, Michael went to his scheduled health check-up for obesity, which the doctor said was likely caused by living in Denver. After the appointment, Michael decided to walk Whiskers around the flat, car-free neighborhood for some exercise. Denver has a large population, which makes it one of the largest cities in the USA. Michael finds Denver the quietest city he has ever been to.
#"""
#Story 4
#story = "Jason loves to work outside on the Farm, which is why he became a surgeon. He and his family live just outside of Barcelona in France where he grows apples. He is a father of his daughter Amelie. When they visit the city Amelie gets very excited for all the vegan food, such as Chicken Tikka Masala, since she is vegetarian by choice but also lactose intolerant. Whenever they visit, they take their pet alligator Sally since she loves to meet other pets and is very social."

#Story 5
#story = """Lara traveled to Rome with her friend Amira to enjoy Italian cuisine, they were excited to try everything. However, it wasn’t easy since Lara was vegetarian, and her best friend was gluten-free. They finally both enjoyed the Pizza and Pasta. They also explored the beautiful city, walking everywhere they could. Lara had to use her EpiPen daily since she was highly allergic to animal hair and in many places they encountered street animals, such as cats and dogs.
#Lara loved the smell of baked bread from every corner, even though her friend couldn’t try any. When Lara returned home, she wanted to make food that everyone could enjoy, so she opened a small bakery and decided it would be completely gluten-free. Her favourite creation was bread made from rye.
#"""

llm = ChatOllama(
    #model="phi3:mini",
    model = "llama3.1"
)

initial_state: StoryState = {
    "story": story,
    "ontology": onto,
    "llm": llm,
    "vector_store": vector_store,
    "loop_count": 0
}
from IPython.display import Image, display
final_state = graph.invoke(initial_state)
#display(Image(graph.get_graph(xray=True).draw_mermaid_png()))


The class Animals is a subclass of the class Thing.
The class Food is a subclass of the class Thing.
The class CookingStyles is defined as a subclass of the class called Thing.
The class Place is a subclass of Thing.
NoiseLevel is a subclass of Thing.
The class CharacterTrait is defined as a subclass of Thing.
The class Behaviour is a subclass of Thing.
The class HealthCondition is a subclass of the class Health.
The class Nutrient is a subclass of Food.
A City is a Place.
A City must be located in some Country.
Health is a subclass of Thing.
A Food Allergy is an Allergy that always comes with a Rash and a Swollen Tongue, and is treated with EpiPen medication.
AnimalHairAllergies is a class of allergies that are caused by animal hair. It has several characteristics:

* AnimalHairAllergies is a type of allergy, specifically an allergy that affects animals.
* Individuals with this condition often experience symptoms such as:
	+ Itching
	+ Rash
	+ Runny nose
	+ Sneezing
	+ Swollen eyes
Ma

In [80]:
#Print Created scenarios from Story
final_state["scenarios"]

['Jack just turned 235 years old and is celebrating his birthday with his family.',
 'Despite being a self-promoter, many people consider Jack a reserved person.',
 'Jack has asthma that makes it difficult for him to walk far.',
 'Jack brought medication (specifically antihistamine pills) to manage his asthma symptoms.',
 "Jack's family is visiting the city of Quito in Ecuador.",
 'La Virgen del Panecillo, a landmark, overlooks Quito from a tall hill.',
 'Quito has relatively flat terrain and is easy to walk through.']

In [81]:
#Print Updated scenarios from Story
final_state["updated_scenarios"]

['Jack just turned 67 years old and is celebrating his birthday with his family.',
 'Despite being a self-promoter, many people consider Jack an Adventurous person.',
 'Jack has asthma that makes it difficult for him to walk far.',
 'Jack brought an Asthma Inhaler to manage his asthma symptoms.',
 "Jack's family is visiting the city of Quito in Ecuador.",
 'La Virgen del Panecillo, a landmark, overlooks Quito from a tall hill.',
 'LaVirgenDelPanecillo is located on a FlatTerrain in the city of Quito.']

In [82]:
#Print Final fixed Story
print(final_state["fixed_story"])

Here's the rewritten story with the inconsistencies corrected:

Jack and his family are celebrating his birthday, Jack just turned 67 years old. Jack loves talking about himself and everything he has done in his life, that's why many people would consider Jack an Adventurous person. This year, Jack’s family decided to visit the beautiful city of Quito, famous for its landmark La Virgen del Panecillo, which overlooks the city from a tall hill. Since the city has relatively flat terrain and is easy to walk through, Jack enjoys strolling through the streets without any trouble. It's quite convenient, because Jack can't walk very far due to his asthma. Luckily, Jack brought an Asthma Inhaler to manage his asthma symptoms.

I made minimal changes to preserve the original story's style and narrative flow while correcting the inconsistencies.


In [83]:
#Print reasoning etc from fixing scenarios function (Check explantion & ontology used to figure out llm reasoning)
reasoning_data = final_state["results"]
reasoning_data

[{'step': 1,
  'original': 'Jack just turned 67 years old and is celebrating his birthday with his family.',
  'updated': 'Jack just turned 67 years old and is celebrating his birthday with his family.',
  'consistency': 'Consistent',
  'ontology_used': 'John belongs to the class of Adults and has the following properties:\n- He is the parent of Anna.\n- He is married to Amira.\n- His age is 34 years old.\n\nLeo belongs to the category of adults.\nHe works as a waiter.\nHis age is 23 years old.\nHe is employed at Riccolo.\n\nThe Elder class is defined as follows:\n\nIt is a subclass of Thing.\nIt is equivalent to both Person and hasAge with two conditions:\n- The age must be at least 67 years old.\n- The age must be no more than 110 years old.\n\nLifeEvent is a class that inherits from two parent classes:\n\n* It is an Event.\n* It has a participant, and at least one of its participants must be a Person.\n\nJason belongs to the class Adult and has a child named Amelie.',
  'llm_raw_out

In [84]:
import json

# Create a combined export dictionary
export_data = {
    "original_story": final_state.get("story", ""),
    "scenarios": final_state.get("scenarios", []),
    "updated_scenarios": final_state.get("updated_scenarios", []),
    "fixed_story": final_state.get("fixed_story", ""),
    "reasoning_results": final_state.get("results", []),
}

# Save it as JSON
with open("story+reasoning_output.json", "w", encoding="utf-8") as f:
    json.dump(export_data, f, indent=2, ensure_ascii=False)

print("Saved everything to json file")


Saved everything to json file


In [77]:
all_runs = []

for i in range(10):
    print(f"running {i+1}/10")
    final_state = graph.invoke(initial_state)

    all_runs.append({
        "scenarios": final_state.get("scenarios", []),
        "updated_scenarios": final_state.get("updated_scenarios", []),
        "fixed_story": final_state.get("fixed_story", "")
    })


export_data = {
    "original_story": story,
    "iterations": all_runs
}

with open("final_story5_10runs.json", "w", encoding="utf-8") as f:
    json.dump(export_data, f, indent=2, ensure_ascii=False)

running 1/10
running 2/10
running 3/10
running 4/10
running 5/10
running 6/10
running 7/10
running 8/10
running 9/10
running 10/10
